In [61]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.grid_search import GridSearchCV
from xgboost.sklearn import XGBRegressor

#training_data = "../Data/train_shuffled_noCat110Cat116_oneHot.csv"
#dataset = pd.read_csv(training_data,delimiter=",", header=0, index_col=0)
#dataset.drop(['id'], axis=1, inplace=True)

#Hier zu mischen und one-hot-encoding einzuführen macht das droppen deutlich einfacher. 
#Die wegfallenden Kategorien wurden weiter unten ermittelt (cat110 und cat116 wurden schon im vorhinen entfehrnt).
dataset = pd.read_csv('../Data/train_noOutliers.csv',delimiter=",", header=0, index_col=0)
testset = pd.read_csv('../Data/test.csv',delimiter=",", header=0)
dataset['identifier']='train'
testset['identifier']='test'

combSet = dataset.append(testset)
#print dataset.columns.values

combSet.drop(['id','cat110','cat116'], axis=1, inplace=True)

combSet = pd.get_dummies(combSet, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=True)

combSet.drop(['cat22_B', 'cat55_B', 'cat63_B', 'cat74_C', 'cat75_C', 'cat88_B', 'cat89_E', 'cat89_G', 'cat89_I', 'cat90_D', 'cat90_E', 'cat90_F', 'cat90_G', 'cat91_H', 'cat92_C', 'cat92_D', 'cat92_F', 'cat92_I', 'cat94_E', 'cat94_G', 'cat96_B', 'cat96_F', 'cat97_B', 'cat97_F', 'cat99_G', 'cat99_H', 'cat99_I', 'cat99_O', 'cat99_P', 'cat99_R', 'cat99_T', 'cat100_E', 'cat101_B', 'cat101_E', 'cat101_H', 'cat101_K', 'cat101_N', 'cat102_G', 'cat103_K', 'cat103_N', 'cat104_Q', 'cat105_B', 'cat105_P', 'cat105_Q', 'cat105_R', 'cat105_S', 'cat106_B', 'cat106_O', 'cat106_P', 'cat106_R', 'cat107_B', 'cat107_R', 'cat107_S', 'cat107_U', 'cat108_C', 'cat108_F', 'cat108_J', 'cat109_AA', 'cat109_AE', 'cat109_AF', 'cat109_AG', 'cat109_AH', 'cat109_AJ', 'cat109_AK', 'cat109_AN', 'cat109_AO', 'cat109_AP', 'cat109_AQ', 'cat109_AR', 'cat109_AT', 'cat109_AU', 'cat109_AV', 'cat109_AW', 'cat109_AY', 'cat109_B', 'cat109_BA', 'cat109_BC', 'cat109_BE', 'cat109_BF', 'cat109_BG', 'cat109_BK', 'cat109_BM', 'cat109_BN', 'cat109_BP', 'cat109_BR', 'cat109_BS', 'cat109_BT', 'cat109_BV', 'cat109_BY', 'cat109_CB', 'cat109_CC', 'cat109_CE', 'cat109_CF', 'cat109_CG', 'cat109_CH', 'cat109_CI', 'cat109_CJ', 'cat109_CK', 'cat109_CL', 'cat109_H', 'cat109_J', 'cat109_K', 'cat109_O', 'cat109_P', 'cat109_Q', 'cat109_V', 'cat109_ZZ', 'cat111_B', 'cat111_D', 'cat111_F', 'cat111_Q', 'cat111_S', 'cat111_Y', 'cat112_AQ', 'cat112_AW', 'cat112_B', 'cat113_AB', 'cat113_AC', 'cat113_AL', 'cat113_AP', 'cat113_AQ', 'cat113_AR', 'cat113_B', 'cat113_BB', 'cat113_BI', 'cat113_BL', 'cat113_E', 'cat113_G', 'cat113_O', 'cat113_P', 'cat113_T', 'cat113_U', 'cat114_B', 'cat114_D', 'cat114_G', 'cat114_S', 'cat114_W', 'cat114_X', 'cat115_B', 'cat115_C', 'cat115_D', 'cat115_E', 'cat115_W', 'cat115_X'], axis=1, inplace=True)

dataset = combSet.loc[combSet['identifier_train']==1.0]
testset = combSet.loc[combSet['identifier_train']==0.0]

dataset = dataset.reindex(np.random.permutation(dataset.index))
dataset.is_copy=False
testset.is_copy=False
dataset.drop('identifier_train',axis=1, inplace=True)
testset.drop(['identifier_train','loss'],axis=1, inplace=True)

print "Breite des letztichen Datensatzes: ", len(dataset.columns)

testSize = 0.1 * len(dataset)
#part = 1
final_train = dataset[0:int(9*testSize)]
#part = 2
final_eval = dataset[int(9*testSize):-1]

#y_train = final_train.pop('loss')
y_train = final_train.pop('loss')
y_eval = final_eval.pop('loss')
#trainDM = xgb.DMatrix(final_train, label=final_train['loss'])
#evalDM = xgb.DMatrix(final_eval, label=final_eval['loss'])
trainDM = xgb.DMatrix(final_train, label=y_train)
evalDM = xgb.DMatrix(final_eval, label=y_eval)
testDM = xgb.DMatrix(testset)

print "Data managment done!"

['id' 'cat1' 'cat2' 'cat3' 'cat4' 'cat5' 'cat6' 'cat7' 'cat8' 'cat9'
 'cat10' 'cat11' 'cat12' 'cat13' 'cat14' 'cat15' 'cat16' 'cat17' 'cat18'
 'cat19' 'cat20' 'cat21' 'cat22' 'cat23' 'cat24' 'cat25' 'cat26' 'cat27'
 'cat28' 'cat29' 'cat30' 'cat31' 'cat32' 'cat33' 'cat34' 'cat35' 'cat36'
 'cat37' 'cat38' 'cat39' 'cat40' 'cat41' 'cat42' 'cat43' 'cat44' 'cat45'
 'cat46' 'cat47' 'cat48' 'cat49' 'cat50' 'cat51' 'cat52' 'cat53' 'cat54'
 'cat55' 'cat56' 'cat57' 'cat58' 'cat59' 'cat60' 'cat61' 'cat62' 'cat63'
 'cat64' 'cat65' 'cat66' 'cat67' 'cat68' 'cat69' 'cat70' 'cat71' 'cat72'
 'cat73' 'cat74' 'cat75' 'cat76' 'cat77' 'cat78' 'cat79' 'cat80' 'cat81'
 'cat82' 'cat83' 'cat84' 'cat85' 'cat86' 'cat87' 'cat88' 'cat89' 'cat90'
 'cat91' 'cat92' 'cat93' 'cat94' 'cat95' 'cat96' 'cat97' 'cat98' 'cat99'
 'cat100' 'cat101' 'cat102' 'cat103' 'cat104' 'cat105' 'cat106' 'cat107'
 'cat108' 'cat109' 'cat110' 'cat111' 'cat112' 'cat113' 'cat114' 'cat115'
 'cat116' 'cont1' 'cont2' 'cont3' 'cont4' 'cont5' 'cont

In [55]:

print final_train
print final_train['loss']
#print testset

           cont1   cont10    cont11    cont12    cont13    cont14     cont2  \
136474  0.585115  0.58257  0.644013  0.630853  0.354344  0.709526  0.488789   
107509  0.513457  0.61459  0.727671  0.714544  0.782450  0.364647  0.358319   
126626  0.536592  0.56081  0.480509  0.481306  0.789182  0.310629  0.488789   
94794   0.351358  0.40666  0.468839  0.458493  0.304350  0.214951  0.737068   
179555  0.455585  0.34928  0.453334  0.443374  0.345247  0.782302  0.422197   
27952   0.338258  0.36458  0.341813  0.335036  0.324464  0.764835  0.620805   
122881  0.899689  0.83510  0.592525  0.579845  0.848129  0.721512  0.620805   
16256   0.465671  0.36458  0.223038  0.230661  0.268941  0.210465  0.159990   
18851   0.667647  0.32935  0.288058  0.282971  0.562997  0.662757  0.299102   
75055   0.057987  0.20496  0.180456  0.178698  0.282249  0.242511  0.488789   
38103   0.533707  0.65072  0.797841  0.785706  0.678452  0.197447  0.555782   
174521  0.513457  0.37493  0.338312  0.338446  0.315

KeyError: 'loss'

In [57]:
param = {'max_depth':3, 'learning_rate':0.1, 'n_estimators':500, 'silent':True, 
         'objective':'reg:linear', 'nthread':10, 'gamma':0, 'min_child_weight':3, 
         'max_delta_step':0, 'subsample':0.7, 'colsample_bytree':0.5, 'colsample_bylevel':1, 
         'reg_alpha':100, 'reg_lambda':1, 'scale_pos_weight':1, 'seed':42, 'missing':None, 'eval_metric':'mae'}

xgb_model = xgb.train(param, trainDM,
num_boost_round=1000, evals=[(evalDM,'eval')], early_stopping_rounds=10, verbose_eval=10)

[0]	eval-mae:2524.13
Will train until eval-mae hasn't improved in 10 rounds.
[10]	eval-mae:1331.37
[20]	eval-mae:1220
[30]	eval-mae:1200.5
[40]	eval-mae:1179.99
[50]	eval-mae:1163
[60]	eval-mae:1147.87
[70]	eval-mae:1134.21
[80]	eval-mae:1123.85
[90]	eval-mae:1115.84
[100]	eval-mae:1109.49
[110]	eval-mae:1103.98
[120]	eval-mae:1100.07
[130]	eval-mae:1095.82
[140]	eval-mae:1093.76
[150]	eval-mae:1090.66
[160]	eval-mae:1088.16
[170]	eval-mae:1085.98
[180]	eval-mae:1084.18
[190]	eval-mae:1082.37
[200]	eval-mae:1081.21
[210]	eval-mae:1079.96
[220]	eval-mae:1078.68
[230]	eval-mae:1077.05
[240]	eval-mae:1076.09
[250]	eval-mae:1075.51
[260]	eval-mae:1074.75
[270]	eval-mae:1074.06
[280]	eval-mae:1073.11
[290]	eval-mae:1072.3
[300]	eval-mae:1071.68
[310]	eval-mae:1071.27
[320]	eval-mae:1070.74
[330]	eval-mae:1070.25
[340]	eval-mae:1069.46
[350]	eval-mae:1068.74
[360]	eval-mae:1068.14
[370]	eval-mae:1067.68
[380]	eval-mae:1067.25
[390]	eval-mae:1066.75
[400]	eval-mae:1066.36
[410]	eval-mae:1065.

In [64]:
predictions = xgb_model.predict(testDM)
for i in range(len(predictions)):
    if predictions[i]>10000:
        print predictions[i]

10532.7
10257.5
10268.0
11381.4
10378.5
10818.9
10504.8
10784.3
10254.9
10187.2
10472.3
10786.4
10047.9
10378.2
10154.7
10109.1
11134.2
10078.3
10262.2
10239.8
10986.4
11038.3
11427.4
10278.5
11109.6
12041.2
10880.5
13537.9
10490.8
10406.2
10515.8
10239.0
10182.9
10958.6
10293.2
10059.8
10138.9
10511.2
10131.6
10868.8
10100.2
11920.2
10884.0
10910.2
10023.5
10070.4
11955.1
10713.0
10243.0
11515.7
11041.6
10383.8
10112.8
10007.0
12020.6
10240.1
11135.4
12051.1
11313.4
10508.1
10877.8
10237.8
10028.8
10556.1
10552.8
10496.2
10210.7
10204.1
10139.3
10201.3
10520.0
10283.2
10485.3
10263.2
10485.4
11004.5
11003.8
10438.7
10690.0
10654.1
10519.2
10212.3
11841.7
10008.4
10554.9
10200.2
10168.9
11081.8
10811.6
12107.6
10220.4
10687.1
10417.5
10828.3
10734.8
12109.1
10768.6
12253.3
10696.2
10404.0
10333.4
12501.5
10114.2
10007.5
10117.8
10583.1
10011.1
10687.8
10512.0
10566.1
11430.3
10126.7
10633.8
10434.4
10647.2
10646.0
11865.9
10417.0
10382.1
10318.4
11022.3
10803.9
10009.0
10555.9
10097.9


In [65]:
testset2 = pd.read_csv('../Data/test.csv',delimiter=",", header=0)
testset2['loss'] = predictions
outFrame = testset2[['id','loss']]
outFrame.to_csv('../Pred/sub1.csv', index=False)